In [853]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np



In [854]:
currency_list = ['USDCHF10080',
                 'GBPUSD10080',
                 #'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 #'GBPCHF10080', 
                 #'EURCHF10080',
                 #'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 #'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                #'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]

all_currency_prediction = []

for q in currency_list:

    currency = q.replace('10080','')
    
    data = pd.read_excel('files/currency_training_data/' + currency +'_combine_data_dataframe.xlsx', sheet_name=0)
    
    # classify each row and attach numeric score to it
    classification_score = []

    # Iterate over each row
    for index, row in data.iterrows():

        # condition when the curreny movement is a buy

        pips_corrector = 100000

        if currency.endswith('jpy'):# use this to correct multiplier of jpy pairs
            pips_corrector = 1000

        if row.nextweek_open < row.nextweek_close:

            trend = 1


            #classifying the risk level


            if (row.nextweek_open - row.nextweek_low) == 0 : # no risk because open is equal to low

                risk = 6

            else : 

                if (row.nextweek_open - row.nextweek_low)> (row.nextweek_high - row.nextweek_open):

                    risk = 1

                if  (row.nextweek_high - row.nextweek_open) > (row.nextweek_open - row.nextweek_low):

                    risk = 2


                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >2 :

                    risk = 3

                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >4 :

                    risk = 4


                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >8 :

                    risk = 5

                if (row.nextweek_high - row.nextweek_open) /(row.nextweek_open - row.nextweek_low) >16 :

                    risk = 6  

            #classifying reward level

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 0 :

                reward = 1

            if (row.nextweek_close - row.nextweek_open)*pips_corrector >200 :

                reward = 2

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 400 :

                reward = 3


            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 800 :

                reward = 4

            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 1600 :

                reward = 5


            if (row.nextweek_close - row.nextweek_open)*pips_corrector > 3200 :

                reward = 6  




        # this is for the condition when the movement is a sell
        else: 

            trend = -1

            #classifying the risk level


            if (row.nextweek_high - row.nextweek_open) == 0 : # no risk because open is equal to high

                risk = 6

            else : 

                if (row.nextweek_high - row.nextweek_open)> (row.nextweek_open - row.nextweek_low):

                    risk = 1

                if  (row.nextweek_open - row.nextweek_low) > (row.nextweek_high - row.nextweek_open):

                    risk = 2


                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >2 :

                    risk = 3

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >4 :

                    risk = 4

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >8 :

                    risk = 5

                if (row.nextweek_open - row.nextweek_low) /(row.nextweek_high - row.nextweek_open) >16 :

                    risk = 6 


             #classifying reward level

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 0 :

                reward = 1

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 200 :

                reward = 2

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 400 :

                reward = 3        

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 800 :

                reward = 4    

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 1600 :

                reward = 5 

            if (row.nextweek_open - row.nextweek_close)*pips_corrector > 3200 :

                reward = 6        

        classification_score.append(trend*risk*reward)



    #attach the classification score value to the dataframe and train a model with the data


    
    data['classification_score'] = classification_score 
    
    



    X = data.drop(columns=['Unnamed: 0', 
                           'date_start',  'nextweek_open', 'nextweek_high','nextweek_low',                       
                            'nextweek_close','classification_score',


                          ])


   

    y = data['classification_score']
    
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # 80% training and 30% test
    
   
    
    
   
  
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.00001 )

    clf=RandomForestClassifier(n_estimators=500,  )

    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,y_train)

    y_pred=clf.predict(X_test)
    #print(y_pred)
    
    

    #temp_data = pd.read_excel('files/combine_top_10_rows_dataframe.xlsx', sheet_name=0)
    
    #temp_data = temp_data.head(1)
    
    
    
    #currency1 =q[ 0 : 3 ].lower()
    #currency2 = q[3:6].lower()
    
   
    
    #index1 = temp_data.columns.get_loc(currency1+'_buy_percent')
    #index2 = temp_data.columns.get_loc(currency2+'_buy_percent')
    
    #temp_data = temp_data.iloc[:, [ 1, index1, index1+1, index1+2, index1+3, index2, index2+1, index2+2, index2+3]   ]
    
    
    
    
    
    
    #last_date = temp_data['date_start'][0].strftime('%Y-%m-%d')
    #X = temp_data.drop(columns=[ 'date_start','Unnamed: 0' ])
   
    
 
    #y_pred = model.predict(X)
    #all_currency_prediction.append(y_pred[0])
    
    print(q)
    
    
  
    # Model Accuracy, how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
    
    feature_imp = pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False)
    #print(feature_imp)
    
    print( len(y_pred),len(y_test) )
    
    custom_pred=[]
    custom_test=[]
    for index, x in enumerate(y_pred):
        if x >23 or x < -23:
            
            custom_pred.append(x)
            custom_test.append(y_test.values.tolist()[index])
            print(x ,'is', y_test.values.tolist()[index])
    
    
    mae = metrics.mean_absolute_error(custom_test, custom_pred)
    mse = metrics.mean_squared_error(y_test, y_pred)
    r2 = metrics.r2_score(y_test, y_pred)

    print("The model performance for testing set")
    print("--------------------------------------")
    print('MAE is {}'.format(mae))
    print('MSE is {}'.format(mse))
    print('R2 score is {}'.format(r2))

print(all_currency_prediction)


USDCHF10080
Accuracy: 0.0273972602739726
73 73
30 is 10
24 is -24
24 is 18
24 is -25
24 is 30
24 is 12
24 is 20
24 is -2
30 is -15
24 is 18
The model performance for testing set
--------------------------------------
MAE is 22.2
MSE is 410.05479452054794
R2 score is -0.806001209961354
GBPUSD10080
Accuracy: 0.0136986301369863
73 73
30 is 20
-30 is 20
-30 is -8
24 is -2
24 is 18
24 is -15
24 is 4
30 is 3
24 is 12
-30 is -2
30 is 2
-25 is 18
The model performance for testing set
--------------------------------------
MAE is 25.916666666666668
MSE is 605.917808219178
R2 score is -1.5100286532951288
[]


In [855]:
 data = {
            'currency':currency_list,
            'score':all_currency_prediction
     
             }
                                     
df = pd.DataFrame(data)

df.to_excel('files/' + last_date + '_' + 'scores.xlsx')

ValueError: All arrays must be of the same length